<a href="https://colab.research.google.com/github/royam0820/HuggingFace/blob/main/amr_llama_code_instr_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2
LLaMA 2 is available in 3 sizes:
- 7b 7 billion parameters
- 13b 13 billion parameters
- 70b 70 billion parameters

This notebook uses the 7b parameters to fine tune our dataset TokenBender/code_instructions_120k_alpaca_style



# Vocabulary

- **LoRA**, which stands for Low-Rank Adapters (LoRA), are small sets of trainable parameters, injected into each layer of the Transformer architecture while fine-tuning. While original model weights are frozen and not updated, these small sets of injected weights are updated during fine-tuning. This greatly reduces the number of trainable parameters for downstream tasks. Gradients during stochastic gradient descent are passed through the frozen pre-trained model weights to the adapter. Thus, only these adapters, with a small memory footprint, are updated during the time of training.
- **Quantization** means “rounding” off values, from one data type to another. It works with squeezing larger values into data types with less number of bits, but with a small loss of precision.


# HuggingFace Support for fine-tuning

HuggingFace has released several libraries that can be used to fine-tune LLMs easily.

These include:

- PEFT Library: HuggingFace has released a library on Parameter Efficient Fine Tuning (PEFT) which has support for LORA.
- Quantization Support — Many models can be loaded in 8-bit and 4-bit precision using bitsandbytes module. The basic way to load a model in 4bit is to pass the argument load_in_4bit=True when calling the from_pretrained method.
- Accelerate library — Accelerate library has many features to make it easy to reduce the memory requirements of models
- Supervised Fine-Tuning Trainer — The SFT trainer is the trainer class for supervised fine-tuning of Large LLMs.

Now we combine all the techniques here to train the llama2-7b on a finance dataset. Notice, we set the storage type to 4-bit and the computation type to FP-16.

# Minimal Code

Here, a 4-bit quantization and PEFT to fine-tune Llama2-7b on a single Google Colab instance!
> Two major components that democratize the training of LLMs are: **Parameter-Efficient Fine-tuning (PEFT)** (e.g: LoRA (Low Rank Adapter)
, Adapter) and **quantization** techniques (8-bit, 4-bit)


![image](https://pbs.twimg.com/media/F1fj-SqWYAELWpM?format=jpg&name=medium)

NB:
- `AutoModelForCausalLM` is used for auto-regressive language models like all the GPT models.
- `SFTTrainer`: [Supervised fine-tuning](https://huggingface.co/docs/trl/main/en/sft_trainer)
( (or SFT for short) is a crucial step in RLHF. In TRL we provide an easy-to-use API to create your SFT models and train them with few lines of code on your dataset.







# Setup

Run the cells below to setup and install the required libraries. For our experiment we will need accelerate, peft, transformers, datasets,scipy and TRL to leverage SFTTrainer. We will use bitsandbytes to quantize the base model into 4bit. We will also install einops but it was mainly used for loading falcon so I will remove it in later versions.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops scipy wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18

NB:
- The `bitsandbytes` is a lightweight wrapper around CUDA custom functions, in particular 8-bit optimizers, matrix multiplication (LLM.int8()), and quantization functions.
- `Quantization` is a technique to reduce the computational and memory costs of running inference by representing the weights and activations with low-precision data types like 8-bit integer (int8) instead of the usual 32-bit floating point (float32).
- `[Accelerate]`(https://huggingface.co/docs/accelerate/index) is a library that enables the same PyTorch code to be run across any distributed configuration by adding just four lines of code! In short, training and inference at scale made simple, efficient and adaptable.
-`einops` stands for Einstein-Inspired Notation for operations (though "Einstein operations" is more attractive and easier to remember).
- `scipy`: SciPy provides algorithms for optimization, integration, interpolation, eigenvalue problems, algebraic equations, differential equations, statistics and many other classes of problems.
- `wandb`:WandB is a central dashboard to keep track of your hyperparameters, system metrics, and predictions so you can compare models live, and share your findings.



# Dataset
https://huggingface.co/datasets/TokenBender/code_instructions_120k_alpaca_style

In [ ]:
from datasets import load_dataset

dataset_name = 'TokenBender/code_instructions_120k_alpaca_style'
dataset = load_dataset(dataset_name, split="train")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/TokenBender___json/TokenBender--code_instructions_120k_alpaca_style-908250f530b0c0f4/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


NB: Datasets are loaded from a dataset loading script that downloads and generates the dataset. However, you can also load a dataset from any dataset repository on the Hub without a loading script!

In [ ]:
# loging to the HF hub to get access to the authentication token
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


# Loading the Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# model_name = "meta-llama/Llama-2-7b-chat-hf"
#if you're running on google colab free tier, uncomment below model and use it instead
model_name = "abhishek/llama-2-7b-hf-small-shards"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

NB: `trust_remote_code=True` is simply saying that it's OK for this model code to be downloaded and run from the hub.

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

NB: `peft`: [Parameter-Efficient Fine-Tuning of Billion-Scale Models on Low-Resource Hardware](https://huggingface.co/blog/peft)}

# Loading the Trainer

Here we will use the SFTTrainer from TRL library that gives a wrapper around transformers Trainer to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 1.4e-4
max_grad_norm = 0.3
max_steps = 200
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/121959 [00:00<?, ? examples/s]

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)


# Train the model
Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,56.619100
20,0.417900
30,9.835900
40,5.235200
50,40.771000
60,11.101600
70,1.483100
80,40.873600
90,13.235500
100,13.415700


TrainOutput(global_step=200, training_loss=28.03741982460022, metrics={'train_runtime': 481.7392, 'train_samples_per_second': 6.643, 'train_steps_per_second': 0.415, 'total_flos': 1.32859924119552e+16, 'train_loss': 28.03741982460022, 'epoch': 0.03})

During QLoRA training, the training losses are spiking and falling sharply. The training loss also drops to zero after 200 steps in my training.

The SFTTrainer will take care of properly saving only the adapters during training instead of saving the entire model.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

# Inference

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

NB: Lora: is a technique that accelerates the fine-tuning of large models while consuming less memory.`lora_config` allows you to control how LoRA is applied to the base model through the following parameters: r : the rank of the update matrices, expressed in int .

In [ ]:
text = '''[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a python program to perform binary search in a given list.[/INST]'''
device = "cuda:0"

In [ ]:
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


[INST]<<SYS>>
 You are a helpful coding assistant that provides code based on the given query in context.
<</SYS>>
Write a python program to perform binary search in a given list.[/INST]

### Solution

```python
def binary_search(arr, target):
    low = 0
    high = len(arr) - 1
    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            high = mid - 1
        else:
            low = mid + 1
    return -1

arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
target = 5

print(binary_search(arr, target))
```

### Explanation

The idea is to find the index of the element in the list that is equal to the given target.

The algorithm is as follows:

1. Initialize the low and high index to 0 and len(arr) - 1 respectively.
2. While the low index is less than the high index, do the following:
    - Calculate the mid index by dividing the low and high index by 2.
    - If the element at the mid index is equal to the 

# Model push to the HF Hub

In [ ]:
model.push_to_hub("llama2-chat-hub-my-finetuned-model")

CommitInfo(commit_url='https://huggingface.co/royam0820/llama2-chat-hub-my-finetuned-model/commit/06ca03e3c078d04668e0cc04621582b433dfec1f', commit_message='Upload model', commit_description='', oid='06ca03e3c078d04668e0cc04621582b433dfec1f', pr_url=None, pr_revision=None, pr_num=None)

# Chat Web UI for Llama

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 7.6 MB/s eta 0:00:00
   

We will import:
- `gradio` is the fastest way to demo your machine learning model with a friendly web interface so that anyone can use it, anywhere!
- `text_generation` is an llm task for producing new text. These models can, for example, fill in incomplete text or paraphrase.

In [ ]:
!pip install text_generation

NB: Text generation works by utilizing algorithms and language models to process input data and generate output text. It involves training AI models on large datasets of text to learn patterns, grammar, and contextual information. These models then use this learned knowledge to generate new text based on given prompts or conditions.

In [ ]:
import os

import gradio as gr
from text_generation import Client


# llama prompt starts with <s> and ends with </s>
# system prompt
PROMPT = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

"""

# os environment variables
LLAMA_70B = os.environ.get("LLAMA_70B", "http://localhost:3000")
CLIENT = Client(base_url=LLAMA_70B)

# creating a dictionary for the LLM parameters
PARAMETERS = {
    "temperature": 0.9,
    "top_p": 0.95,
    "repetition_penalty": 1.2,
    "top_k": 50,
    "truncate": 1000,
    "max_new_tokens": 1024,
    "seed": 42,
    "stop_sequences": ["</s>"],
}

# Message formatting
def format_message(message, history, memory_limit=5):
    # handling the context, keeping 5 last messages in memory
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return PROMPT + f"{message} [/INST]"

    formatted_message = PROMPT + f"{history[0][0]} [/INST] {history[0][1]} </s>"

    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

# Inference
# message is the current user query, history are the past user's queries
def predict(message, history):
    query = format_message(message, history)
    text = "" #it will hold the response text
    for response in CLIENT.generate_stream(query, **PARAMETERS):
        if not response.token.special:
            text += response.token.text
            yield text

# lauching the Gradio webui chat interface
gr.ChatInterface(predict).queue().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d757f411ac7210a7d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
